# Reference

*   Original Tutorial: http://mlexplained.com/2019/01/30/an-in-depth-tutorial-to-allennlp-from-basics-to-elmo-and-bert/
*   Original Colab: https://colab.research.google.com/github/dudeperf3ct/DL_notebooks/blob/master/tl_nlp/tl_nlp_allennlp.ipynb#scrollTo=pDBy4oPBOftc
*   Modified ELMO Colab for text classification: https://github.com/keitakurita/Practical_NLP_in_PyTorch/blob/master/allennlp/elmo_text_classification.ipynb

#### Other references:

*   http://www.realworldnlpbook.com/blog/improving-sentiment-analyzer-using-elmo.html
*   https://dudeperf3ct.github.io/nlp/transfer/learning/2019/02/22/Power-of-Transfer-Learning-in-NLP/#elmo
*   https://allennlp.org/tutorials


In [ ]:
!pip install -q allennlp==0.8.2

import allennlp
allennlp.__version__

     |████████████████████████████████| 5.6MB 16.2MB/s 
     |████████████████████████████████| 92kB 12.7MB/s 
     |████████████████████████████████| 12.6MB 242kB/s 
     |████████████████████████████████| 317kB 59.7MB/s 
     |████████████████████████████████| 133kB 59.7MB/s 
     |████████████████████████████████| 552kB 59.2MB/s 
     |████████████████████████████████| 133kB 68.0MB/s 
     |████████████████████████████████| 25.2MB 130kB/s 
     |████████████████████████████████| 3.3MB 51.1MB/s 
     |████████████████████████████████| 51kB 8.6MB/s 
     |████████████████████████████████| 245kB 62.7MB/s 
     |████████████████████████████████| 512kB 64.6MB/s 
     |████████████████████████████████| 71kB 5.9MB/s 
     |████████████████████████████████| 4.5MB 49.6MB/s 
     |████████████████████████████████| 133kB 60.8MB/s 
     |████████████████████████████████| 51kB 8.0MB/s 
     |████████████████████████████████| 2.6MB 50.7MB/s 
     |████████████████████████████████| 61kB 8.9MB/s 
 

'0.8.2'

In [ ]:
import spacy
spacy.__version__

'2.0.18'

In [ ]:
import torch
torch.__version__

'1.6.0+cu101'

In [ ]:
!nvidia-smi

Mon Aug 31 11:55:24 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
torch.cuda.set_device(0)

In [ ]:
# https://spacy.io/models/pt

! python -m spacy download pt
! python -m spacy download pt_core_news_sm

cur_language = 'pt_core_news_sm'

     |████████████████████████████████| 38.7MB 737kB/s 
  Created wheel for pt-core-news-sm: filename=pt_core_news_sm-2.0.0-cp36-none-any.whl size=38749345 sha256=387318138119d3a5c28d50f020b2208cdb6954a67f9025c8fabc46125839fb4c
  Stored in directory: /tmp/pip-ephem-wheel-cache-dsg8srxu/wheels/0b/f1/54/a8f759b41cf39d2a33d6da84cd90b8e2fefea549963396bbd2
Successfully built pt-core-news-sm

    Linking successful
    /usr/local/lib/python3.6/dist-packages/pt_core_news_sm -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/pt

    You can now load the model via spacy.load('pt')


    Linking successful
    /usr/local/lib/python3.6/dist-packages/pt_core_news_sm -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/pt_core_news_sm

    You can now load the model via spacy.load('pt_core_news_sm')



## e-SIC Dataset

Code Adapted from : [Link](https://github.com/keitakurita/Practical_NLP_in_PyTorch)

Paper ELMo : [Link](https://arxiv.org/pdf/1802.05365.pdf)


In [ ]:
from pathlib import Path
from typing import *
import os
import torch
import torch.optim as optim
import numpy as np
import pandas as pd
from functools import partial
from overrides import overrides

from allennlp.data import Instance
from allennlp.data.token_indexers import TokenIndexer
from allennlp.data.tokenizers import Token
from allennlp.nn import util as nn_util
from allennlp.common.checks import ConfigurationError

from datetime import datetime

USE_GPU = torch.cuda.is_available()
print(USE_GPU)

True


In [ ]:
import pandas as pd

#Variables
params = {
    'exp': 'Atendimento-Balanced-Multiclass',
    'data': 'resp-text',
    'label': 'Atendimento',
    'BATCH_SIZE': 64,
    'MAX_LEN': 128,
    'lr': 3e-4,
    'epochs': 10,
    'hidden_sz': 64,
    'max_vocab_size': 10000
}

In [ ]:
from google.colab import drive

# Mount Driver
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
# Load Data
binary = False
exp = params.get('exp')
if 'Binary' in exp:
  binary = True

base_path = '/content/gdrive/My Drive/Colab Notebooks/Simple/Datasets/' + exp
save_path = '/content/gdrive/My Drive/Colab Notebooks/Simple/' + exp + '/output/'

data = params.get('data')
label = params.get('label')

x_train_file = 'X_train.csv'
y_train_file = 'y_train.csv'
x_test_file = 'X_test.csv'
y_test_file = 'y_test.csv'

#Load data
X_train = pd.read_csv(os.path.join(base_path, x_train_file), sep=';', encoding='utf-8')
y_train = pd.read_csv(os.path.join(base_path, y_train_file), sep=';', encoding='utf-8')
X_test = pd.read_csv(os.path.join(base_path, x_test_file), sep=';', encoding='utf-8')
y_test = pd.read_csv(os.path.join(base_path, y_test_file), sep=';', encoding='utf-8')

In [ ]:
#df = pd.concat([X_train, X_test], axis=0)
train_df = X_train.loc[:,['pid', data, label]]
test_df = X_test.loc[:,['pid', data, label]]

train_df['positive'] = train_df.apply(lambda row: 1 if row[label] == 2 else 0, axis=1)
train_df['negative'] = train_df.apply(lambda row: 1 if row[label] == 0 else 0, axis=1)
train_df['neutral'] = train_df.apply(lambda row: 1 if row[label] == 1 else 0, axis=1)

test_df['positive'] = test_df.apply(lambda row: 1 if row[label] == 2 else 0, axis=1)
test_df['negative'] = test_df.apply(lambda row: 1 if row[label] == 0 else 0, axis=1)
test_df['neutral'] = test_df.apply(lambda row: 1 if row[label] == 1 else 0, axis=1)

train_df.head(10)

,pid,resp-text,Atendimento,positive,negative,neutral
0,529374,"Prezado Senhor , Em consideração a sua manifes...",1,0,0,1
1,517464,"Prezado a Senhor a , Em atendimento ao pedido ...",1,0,0,1
2,406897,"Prezado , Ao cumprimentá-lo cordialmente , inf...",0,0,1,0
3,388221,"Prezado Senhor , Em atenção à sua demanda regi...",0,0,1,0
4,451583,"Prezado Sr . Yuri , boa tarde , Em atenção a s...",1,0,0,1
5,505170,"Prezado , Segue resposta da Diretoria de Estud...",1,0,0,1
6,354638,Respondente : Serviço de Informação ao Cidadã...,2,1,0,0
7,526968,"Prezado senhor , Conforme informações prestada...",2,1,0,0
8,552107,"Boa tarde , Recebemos , da Pró-Reitoria de Ges...",1,0,0,1
9,403296,"Senhor Gutemberg , O Serviço de Informações ao...",2,1,0,0


In [ ]:
test_df.head(10)

,pid,resp-text,Atendimento,positive,negative,neutral
0,514368,Prezado senhor esclarecemos que Susep não poss...,1,0,0,1
1,452197,Com relação ao pedido de informação de protoc...,1,0,0,1
2,533396,"Prezado Senhor , Em atenção seu pedido formula...",1,0,0,1
3,567521,"Prezada solicitante , Segue , em anexo , respo...",2,1,0,0
4,295701,"Senhor Bernardo, O Serviço de Informações ao C...",1,0,0,1
5,478858,"Prezado Pedro , A publicação é feita nas segui...",2,1,0,0
6,562079,"Prezada Senhora , boa tarde ! Em atenção à su...",1,0,0,1
7,393850,"Prezada Senhora GiovannaTrevizan Benigno , Rec...",1,0,0,1
8,533823,Segue anexo resposta à solicitação de informaç...,1,0,0,1
9,497668,"Senhor Rodrigo , O Serviço de Informações ao C...",2,1,0,0


In [ ]:
try:
  os.mkdir('data/')
except:
  pass

train_df.to_csv('data/train_df.csv', index=False)
test_df.to_csv('data/test_df.csv', index=False)

In [ ]:
class Config(dict):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        for k, v in kwargs.items():
            setattr(self, k, v)
    
    def set(self, key, val):
        self[key] = val
        setattr(self, key, val)
        
config = Config(
    testing=False,
    seed=1,
    batch_size=params.get('BATCH_SIZE'),
    lr=params.get('lr'),
    epochs=params.get('epochs'),
    hidden_sz=params.get('hidden_sz'),
    max_seq_len=params.get('MAX_LEN'), # necessary to limit memory usage
    max_vocab_size=params.get('max_vocab_size'),
)

In [ ]:
torch.manual_seed(config.seed)
DATA_ROOT = Path("data")

### Prepare Dataset

In [ ]:
from allennlp.data.vocabulary import Vocabulary
from allennlp.data.dataset_readers import DatasetReader

In [ ]:
label_cols = ["negative", "neutral", "positive"]

In [ ]:
from allennlp.data.fields import TextField, MetadataField, ArrayField

class SentimentDatasetReader(DatasetReader):
    def __init__(self, tokenizer: Callable[[str], List[str]]=lambda x: x.split(),
                 token_indexers: Dict[str, TokenIndexer] = None,
                 max_seq_len: Optional[int]=config.max_seq_len) -> None:
        super().__init__(lazy=False)
        self.tokenizer = tokenizer
        self.token_indexers = token_indexers or {"tokens": SingleIdTokenIndexer()}
        self.max_seq_len = max_seq_len

    @overrides
    def text_to_instance(self, tokens: List[Token], id: str=None, labels: np.ndarray=None) -> Instance:
        sentence_field = TextField(tokens, self.token_indexers)
        fields = {"tokens": sentence_field}
        
        id_field = MetadataField(id)
        fields["id"] = id_field
        
        if labels is None:
            labels = np.zeros(len(label_cols))
        label_field = ArrayField(array=labels)
        fields["label"] = label_field

        return Instance(fields)
    
    @overrides
    def _read(self, file_path: str) -> Iterator[Instance]:
        df = pd.read_csv(file_path)
        if config.testing: df = df.head(1000)
        for i, row in df.iterrows():
            yield self.text_to_instance([Token(x) for x in self.tokenizer(row[data])], None, row[label_cols].values)

## ELMo

In [ ]:
from allennlp.data.tokenizers.word_splitter import SpacyWordSplitter
from allennlp.data.token_indexers.elmo_indexer import ELMoCharacterMapper, ELMoTokenCharactersIndexer

# the token indexer is responsible for mapping tokens to integers
token_indexer = ELMoTokenCharactersIndexer()

def tokenizer(x: str):
    return [w.text for w in SpacyWordSplitter(language=cur_language, pos_tags=False).split_words(x)[:config.max_seq_len]]

In [ ]:
reader = SentimentDatasetReader(
    tokenizer=tokenizer,
    token_indexers={"tokens": token_indexer}
)

In [ ]:
train_ds = reader.read(DATA_ROOT / "train_df.csv")
print(len(train_ds))

6982it [00:19, 365.62it/s]

6982


In [ ]:
test_ds = reader.read(DATA_ROOT / "test_df.csv")
print(len(test_ds))

2993it [00:07, 394.30it/s]

2993


### Checking Tokens & Labels

In [ ]:
vars(train_ds[0].fields["tokens"])

{'_indexed_tokens': None,
 '_indexer_name_to_indexed_token': None,
 '_token_indexers': {'tokens': <allennlp.data.token_indexers.elmo_indexer.ELMoTokenCharactersIndexer at 0x7f5e6d09e4e0>},
 'tokens': [Prezado,
  Senhor,
  ,,
  Em,
  consideração,
  a,
  sua,
  manifestação,
  registrada,
  no,
  e,
  -,
  SIC,
  ,,
  protocolo,
  número,
  53850000689201784,
  ,,
  informamos,
  que,
  este,
  não,
  é,
  o,
  meio,
  adequado,
  para,
  Vossa,
  Senhoria,
  solicitar,
  recebimento,
  de,
  senha,
  do,
  CADSEI,
  .,
  Por,
  oportuno,
  ,,
  identificamos,
  que,
  Vossa,
  Senhoria,
  também,
  registrou,
  no,
  Sistema,
  de,
  Ouvidorias,
  do,
  Poder,
  Executivo,
  Federal,
  e,
  -,
  Ouv,
  uma,
  demanda,
  com,
  o,
  mesmo,
  teor,
  ,,
  através,
  do,
  protocolo,
  número,
  01217,
  .,
  003492,
  2017,
  -,
  79,
  ,,
  que,
  n,
  este,
  momento,
  reiteramos,
  as,
  orientações,
  contidas,
  naresposta,
  enviada,
  .,
  O,
  Ministério,
  agradece,
  o,
  seu,

In [ ]:
vars(train_ds[0].fields["label"])

{'array': array([0, 1, 0], dtype=object), 'padding_value': 0}

### Prepare Vocabulary

In [ ]:
vocab = Vocabulary()

### Prepare Iterator

In [ ]:
from allennlp.data.iterators import BucketIterator

iterator = BucketIterator(batch_size=config.batch_size, sorting_keys=[("tokens", "num_tokens")],)
iterator.index_with(vocab)

### Prepare Model

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

from allennlp.modules.seq2vec_encoders import Seq2VecEncoder, PytorchSeq2VecWrapper
from allennlp.nn.util import get_text_field_mask
from allennlp.models import Model
from allennlp.modules.text_field_embedders import TextFieldEmbedder

/usr/local/lib/python3.6/dist-packages/sklearn/utils/linear_assignment_.py:22: FutureWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)


In [ ]:
class BaselineModel(Model):
    def __init__(self, word_embeddings: TextFieldEmbedder,
                 encoder: Seq2VecEncoder,
                 out_sz: int=len(label_cols)):
        super().__init__(vocab)
        self.word_embeddings = word_embeddings
        self.encoder = encoder
        self.projection = nn.Linear(self.encoder.get_output_dim(), out_sz)
        self.loss = nn.BCEWithLogitsLoss()
        
    def forward(self, tokens: Dict[str, torch.Tensor],
                id: Any, label: torch.Tensor) -> torch.Tensor:
        mask = get_text_field_mask(tokens)
        embeddings = self.word_embeddings(tokens)
        state = self.encoder(embeddings, mask)
        class_logits = self.projection(state)
        
        output = {"class_logits": class_logits}
        output["loss"] = self.loss(class_logits, label)

        return output

### Prepare Embeddings

In [ ]:
from allennlp.modules.text_field_embedders import BasicTextFieldEmbedder
from allennlp.modules.token_embedders import ElmoTokenEmbedder

# https://allennlp.org/elmo - PORTUGUESE BRWAC
options_file = 'https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/contributed/pt/brwac/options.json'
weight_file = 'https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/contributed/pt/brwac/elmo_pt_weights_dgx1.hdf5'

elmo_embedder = ElmoTokenEmbedder(options_file, weight_file)
word_embeddings = BasicTextFieldEmbedder({"tokens": elmo_embedder})

100%|██████████| 374434792/374434792 [00:27<00:00, 13397153.47B/s]


In [ ]:
from allennlp.modules.seq2vec_encoders import PytorchSeq2VecWrapper

# Create Encoder
encoder: Seq2VecEncoder = PytorchSeq2VecWrapper(nn.LSTM(word_embeddings.get_output_dim(), config.hidden_sz, bidirectional=True, batch_first=True))

In [ ]:
model = BaselineModel(word_embeddings, encoder)

if USE_GPU: model.cuda()
else: model

### Train

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=config.lr)

In [ ]:
from allennlp.training.trainer import Trainer

trainer = Trainer(
    model=model,
    optimizer=optimizer,
    iterator=iterator,
    train_dataset=train_ds,
    cuda_device=0 if USE_GPU else -1,
    num_epochs=config.epochs,
)

In [ ]:
metrics = trainer.train()

loss: 0.6419 ||: 100%|██████████| 110/110 [01:22<00:00,  1.33it/s]
loss: 0.6308 ||: 100%|██████████| 110/110 [01:25<00:00,  1.29it/s]
loss: 0.6258 ||: 100%|██████████| 110/110 [01:27<00:00,  1.26it/s]
loss: 0.6202 ||: 100%|██████████| 110/110 [01:27<00:00,  1.26it/s]
loss: 0.6162 ||: 100%|██████████| 110/110 [01:27<00:00,  1.26it/s]
loss: 0.6107 ||: 100%|██████████| 110/110 [01:27<00:00,  1.26it/s]
loss: 0.6069 ||: 100%|██████████| 110/110 [01:27<00:00,  1.26it/s]
loss: 0.6015 ||: 100%|██████████| 110/110 [01:27<00:00,  1.26it/s]
loss: 0.5961 ||: 100%|██████████| 110/110 [01:27<00:00,  1.26it/s]
loss: 0.5922 ||: 100%|██████████| 110/110 [01:27<00:00,  1.26it/s]


### Predictions

In [ ]:
from allennlp.data.iterators import DataIterator
from tqdm import tqdm
from scipy.special import expit # the sigmoid function

def tonp(tsr): return tsr.detach().cpu().numpy()

class Predictor:
    def __init__(self, model: Model, iterator: DataIterator,
                 cuda_device: int=-1) -> None:
        self.model = model
        self.iterator = iterator
        self.cuda_device = cuda_device
        
    def _extract_data(self, batch) -> np.ndarray:
        out_dict = self.model(**batch)
        return expit(tonp(out_dict["class_logits"]))
    
    def predict(self, ds: Iterable[Instance]) -> np.ndarray:
        pred_generator = self.iterator(ds, num_epochs=1, shuffle=False)
        self.model.eval()
        pred_generator_tqdm = tqdm(pred_generator,
                                   total=self.iterator.get_num_batches(ds))
        preds = []
        with torch.no_grad():
            for batch in pred_generator_tqdm:
                batch = nn_util.move_to_device(batch, self.cuda_device)
                preds.append(self._extract_data(batch))
        return np.concatenate(preds, axis=0)

In [ ]:
from allennlp.data.iterators import BasicIterator

# iterate over the dataset without changing its order
seq_iterator = BasicIterator(batch_size=64)
seq_iterator.index_with(vocab)

In [ ]:
predictor = Predictor(model, seq_iterator, cuda_device=0 if USE_GPU else -1)
test_preds = predictor.predict(test_ds)

100%|██████████| 47/47 [00:43<00:00,  1.09it/s]


In [ ]:
# Convert to predictions
y_pred_bool = np.argmax(test_preds, axis=1)
y_pred_bool[0:10]

array([0, 2, 1, 2, 1, 0, 0, 0, 2, 2])

In [ ]:
from sklearn.metrics import f1_score, classification_report

test_y = test_df[label]

f1 = f1_score(test_y, y_pred_bool, average='weighted')
print(f"Best Test F1-Score: {f1:.3f}")    
print(classification_report(test_y, y_pred_bool))

Best Test F1-Score: 0.388
              precision    recall  f1-score   support

           0       0.39      0.49      0.44       958
           1       0.35      0.19      0.25      1006
           2       0.44      0.53      0.48      1029

    accuracy                           0.40      2993
   macro avg       0.39      0.40      0.39      2993
weighted avg       0.39      0.40      0.39      2993



In [ ]:
now = datetime.now().strftime("%Y%m%d_%H%M%S")
output_dir = '/content/gdrive/My Drive/Colab Notebooks/outputs/elmo/'  + params.get('exp') + '/' + now 

try:
  os.mkdir(output_dir)
except:
  pass

from json import dumps

with open(output_dir + '/params.json', 'w') as f:
  f.write(dumps(params))

In [ ]:
with open(output_dir + "/model.th", 'wb') as f:
    torch.save(model.state_dict(), f)

vocab.save_to_files(output_dir + "/vocabulary")

In [ ]:
with open(output_dir + '/classification_report.txt', 'w') as f:
  f.write(classification_report(test_y, y_pred_bool))